In [37]:
import pandas as pd
import os
import json

from organizeEventData import *
from extract_game_info import *


In [38]:
country = 'italy'

In [39]:
matchesFile = open(f"data/matches/matches_{country}.json")
matches = json.load(matchesFile)
matchesMap = {}
for match in matches:
    matchesMap[match["wyId"]] = match
matchesFile.close()

teamsFile = open("data/teams.json")
teams = json.load(teamsFile)
teamsFile.close()

playersFile = open("data/players.json")
players = json.load(playersFile)
playerMap = {}
for player in players:
    playerMap[player["wyId"]] =player
playersFile.close()

eventsFile = open(f"data/events/events_{country}.json")
events = json.load(eventsFile)
eventsFile.close()

In [40]:
df_matches = pd.read_json(f"/Users/tw/Desktop/Deep_Learning/Project/data/matches/matches_{country}.json")[['wyId', 'label', 'teamsData', 'winner']]

In [41]:
def clean_team_names(team):
    return team_dict[team]

def extract_label_information(label):
    teams, score = label.split(',')
    home_team, away_team = [x for x in teams.split(' - ')]
    home_goals, away_goals = [int(x.replace(' ', '')) for x in score.split(' - ')]

    return home_team, away_team, home_goals, away_goals

def get_team_ids(teamsData):
    return list(teamsData.keys())

def extract_game_information(teamsData, teams):
    teamIds = list(teamsData.keys())
    
    for t_id in teamIds:
        if teamsData[t_id]['side'] == 'home':
            team1_id = t_id
            team1_name = teamNameFromId(int(t_id), teams)
            team1_goals = teamsData[t_id]['score']
        elif teamsData[t_id]['side'] == 'away': 
            team2_id = t_id
            team2_name = teamNameFromId(int(t_id), teams)
            team2_goals = teamsData[t_id]['score']
        else:
            print('ERROR: team ids')
    return team1_id, team1_name, team2_id, team2_name, team1_goals, team2_goals

df_matches['team1_id'], df_matches['team1_name'], df_matches['team2_id'], df_matches['team2_name'], df_matches['team1_goals'], df_matches['team2_goals'] = zip(*df_matches.apply(lambda x: extract_game_information(x['teamsData'], teams), axis='columns'))
# df_matches['team1_id'], df_matches['team2_id'] = zip(*df_matches.apply(lambda x: get_team_ids(x['teamsData']), axis='columns'))

In [42]:
df_matches['team1_win'] = df_matches.apply(lambda x: int(x['winner']) == int(x['team1_id']), axis='columns')
df_matches['team2_win'] = df_matches.apply(lambda x: int(x['winner']) == int(x['team2_id']), axis='columns')

In [43]:
df_matches = df_matches.drop(columns=['label', 'teamsData', 'winner'])
df_matches = df_matches.rename(columns={"wyId": "game_id"})

In [44]:
df_matches.sort_values(by='game_id')

,game_id,team1_id,team1_name,team2_id,team2_name,team1_goals,team2_goals,team1_win,team2_win
377,2575959,3172,Atalanta,3158,Roma,0,1,False,True
370,2575960,3166,Bologna,3185,Torino,1,1,False,False
371,2575961,3197,Crotone,3157,Milan,0,3,False,True
378,2575962,3194,Hellas Verona,3187,Napoli,1,3,False,True
372,2575963,3161,Internazionale,3176,Fiorentina,3,0,True,False
...,...,...,...,...,...,...,...,...,...
9,2576334,3159,Juventus,3194,Hellas Verona,2,1,True,False
0,2576335,3162,Lazio,3161,Internazionale,2,3,False,True
1,2576336,3315,Sassuolo,3158,Roma,0,1,False,True
7,2576337,3204,SPAL,3164,Sampdoria,3,1,True,False


In [45]:
eventsPerMatch = groupEventsByMatch(events)

Done 10 matches
Done 20 matches
Done 30 matches
Done 40 matches
Done 50 matches
Done 60 matches
Done 70 matches
Done 80 matches
Done 90 matches
Done 100 matches
Done 110 matches
Done 120 matches
Done 130 matches
Done 140 matches
Done 150 matches
Done 160 matches
Done 170 matches
Done 180 matches
Done 190 matches
Done 200 matches
Done 210 matches
Done 220 matches
Done 230 matches
Done 240 matches
Done 250 matches
Done 260 matches
Done 270 matches
Done 280 matches
Done 290 matches
Done 300 matches
Done 310 matches
Done 320 matches
Done 330 matches
Done 340 matches
Done 350 matches
Done 360 matches
Done 370 matches
Done 380 matches


In [46]:
# list of dataframe rows (one per game)
game_events_rows = []

for row in df_matches.iterrows():
    match_id = row[1].game_id
    team1_id = row[1].team1_id
    team2_id = row[1].team2_id

    team1Events, team2Events = splitEventsByType(eventsPerMatch[match_id], [int(team1_id), int(team2_id)])

    team1NumDict = calcNumByEventType(team1Events)
    team2NumDict = calcNumByEventType(team2Events)

    team1_row = pd.json_normalize(team1NumDict, sep='_')
    team1_row = team1_row.add_prefix('team1_')
    team2_row = pd.json_normalize(team2NumDict, sep='_')
    team2_row = team2_row.add_prefix('team2_')

    game_row = pd.concat([team1_row, team2_row], axis=1)
    game_row['game_id'] = match_id
    game_events_rows.append(game_row)
    

In [47]:
game_events = pd.concat(game_events_rows, ignore_index=True)

In [48]:
df_matches = df_matches.merge(game_events, on='game_id')

In [49]:
df_matches

,game_id,team1_id,team1_name,team2_id,team2_name,team1_goals,team2_goals,team1_win,team2_win,team1_shot_5,...,team2_averagePassLength_50,team2_averagePassLength_55,team2_averagePassLength_60,team2_averagePassLength_65,team2_averagePassLength_70,team2_averagePassLength_75,team2_averagePassLength_80,team2_averagePassLength_85,team2_averagePassLength_90,team2_averagePassLength_90+
0,2576335,3162,Lazio,3161,Internazionale,2,3,False,True,0,...,24.485578,28.057944,23.722610,26.667222,25.837245,28.349498,27.631560,29.889039,20.469223,21.807284
1,2576336,3315,Sassuolo,3158,Roma,0,1,False,True,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2576329,3173,Cagliari,3172,Atalanta,1,0,True,False,0,...,24.037338,27.976034,26.702997,23.313865,30.484495,23.834016,23.423180,23.248893,28.366314,24.307511
3,2576330,3165,Chievo,3219,Benevento,1,0,True,False,0,...,24.437178,21.383057,21.593970,23.000201,22.776704,23.801477,23.571837,22.910578,27.238948,24.446687
4,2576331,3163,Udinese,3166,Bologna,1,0,True,False,0,...,21.286475,28.788916,23.276116,22.987720,22.627458,20.203752,22.887248,23.017847,19.316512,33.495381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2575967,3315,Sassuolo,3193,Genoa,0,0,False,False,2,...,22.945973,21.354319,22.278223,23.904008,23.856341,29.320651,25.030822,29.356753,25.875036,26.365318
376,2575968,3163,Udinese,3165,Chievo,1,2,False,True,1,...,17.706644,27.046147,22.097327,22.347458,19.591610,19.064234,20.356348,13.284922,26.320393,27.533136
377,2575959,3172,Atalanta,3158,Roma,0,1,False,True,1,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
378,2575962,3194,Hellas Verona,3187,Napoli,1,3,False,True,0,...,21.582762,21.460480,18.361936,19.891525,18.184871,21.658211,23.340103,22.544754,19.788647,23.463808


In [50]:
df_matches.sort_values(by=['game_id'])

,game_id,team1_id,team1_name,team2_id,team2_name,team1_goals,team2_goals,team1_win,team2_win,team1_shot_5,...,team2_averagePassLength_50,team2_averagePassLength_55,team2_averagePassLength_60,team2_averagePassLength_65,team2_averagePassLength_70,team2_averagePassLength_75,team2_averagePassLength_80,team2_averagePassLength_85,team2_averagePassLength_90,team2_averagePassLength_90+
377,2575959,3172,Atalanta,3158,Roma,0,1,False,True,1,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
370,2575960,3166,Bologna,3185,Torino,1,1,False,False,1,...,30.560704,23.701412,21.095795,23.644778,26.764134,16.372781,26.406895,25.769032,24.537232,21.045808
371,2575961,3197,Crotone,3157,Milan,0,3,False,True,0,...,19.797615,16.201426,20.586859,23.149955,23.052535,25.086776,27.139089,21.367198,24.603859,22.996558
378,2575962,3194,Hellas Verona,3187,Napoli,1,3,False,True,0,...,21.582762,21.460480,18.361936,19.891525,18.184871,21.658211,23.340103,22.544754,19.788647,23.463808
372,2575963,3161,Internazionale,3176,Fiorentina,3,0,True,False,0,...,25.059415,30.045641,24.542840,24.999937,26.412254,30.426573,28.270499,27.721299,22.617753,25.038108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,2576334,3159,Juventus,3194,Hellas Verona,2,1,True,False,2,...,25.519687,31.251505,21.407313,18.880532,22.495837,18.345308,21.639595,25.619733,26.232859,29.598672
0,2576335,3162,Lazio,3161,Internazionale,2,3,False,True,0,...,24.485578,28.057944,23.722610,26.667222,25.837245,28.349498,27.631560,29.889039,20.469223,21.807284
1,2576336,3315,Sassuolo,3158,Roma,0,1,False,True,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,2576337,3204,SPAL,3164,Sampdoria,3,1,True,False,2,...,20.110843,18.397336,18.543724,20.421676,23.525321,20.823186,23.910118,22.417150,19.994541,20.055936


In [51]:
# def getStartingLineupAverageHeight(match, playerMap):

#     teamIds = list(match["teamsData"].keys())
#     for t_id in teamIds:
#         if match["teamsData"][t_id]['side'] == 'home':
#             team1_id = t_id
#         else:
#             team2_id = t_id

#     team1Height = 0
#     team2Height = 0
#     for team in teamIds:
#         for player in match["teamsData"][team]["formation"]["lineup"]:
#             playerId = player["playerId"]
#             if (team == team1_id):
#                 team1Height+= playerMap[playerId]["height"]
#             elif (team == team2_id):
#                 team2Height+= playerMap[playerId]["height"]

#     return [team1_id, team1Height/11, team2_id, team2Height/11]

match_list = []
for matchId in df_matches['game_id']:
    match = [matchId] + getStartingLineupAverageHeight(matchesMap[matchId], playerMap)
    match_list.append(match)

In [52]:
df_height = pd.DataFrame(match_list, columns=['game_id', 'team1_id', 'team1_height', 'team2_id', 'team2_height'])

In [53]:
df_height

,game_id,team1_id,team1_height,team2_id,team2_height
0,2576335,3162,184.363636,3161,183.454545
1,2576336,3315,182.818182,3158,186.454545
2,2576329,3173,182.363636,3172,184.090909
3,2576330,3165,182.363636,3219,181.636364
4,2576331,3163,185.545455,3166,185.454545
...,...,...,...,...,...
375,2575967,3315,182.090909,3193,183.000000
376,2575968,3163,186.090909,3165,183.090909
377,2575959,3172,184.272727,3158,184.363636
378,2575962,3194,179.818182,3187,182.636364


In [54]:
df_matches = df_matches.merge(df_height, on=['game_id', 'team1_id', 'team2_id'])

In [55]:
df_matches.sort_values(by=['game_id'])

,game_id,team1_id,team1_name,team2_id,team2_name,team1_goals,team2_goals,team1_win,team2_win,team1_shot_5,...,team2_averagePassLength_60,team2_averagePassLength_65,team2_averagePassLength_70,team2_averagePassLength_75,team2_averagePassLength_80,team2_averagePassLength_85,team2_averagePassLength_90,team2_averagePassLength_90+,team1_height,team2_height
377,2575959,3172,Atalanta,3158,Roma,0,1,False,True,1,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,184.272727,184.363636
370,2575960,3166,Bologna,3185,Torino,1,1,False,False,1,...,21.095795,23.644778,26.764134,16.372781,26.406895,25.769032,24.537232,21.045808,183.545455,180.818182
371,2575961,3197,Crotone,3157,Milan,0,3,False,True,0,...,20.586859,23.149955,23.052535,25.086776,27.139089,21.367198,24.603859,22.996558,184.000000,183.363636
378,2575962,3194,Hellas Verona,3187,Napoli,1,3,False,True,0,...,18.361936,19.891525,18.184871,21.658211,23.340103,22.544754,19.788647,23.463808,179.818182,182.636364
372,2575963,3161,Internazionale,3176,Fiorentina,3,0,True,False,0,...,24.542840,24.999937,26.412254,30.426573,28.270499,27.721299,22.617753,25.038108,182.454545,183.727273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,2576334,3159,Juventus,3194,Hellas Verona,2,1,True,False,2,...,21.407313,18.880532,22.495837,18.345308,21.639595,25.619733,26.232859,29.598672,182.181818,181.454545
0,2576335,3162,Lazio,3161,Internazionale,2,3,False,True,0,...,23.722610,26.667222,25.837245,28.349498,27.631560,29.889039,20.469223,21.807284,184.363636,183.454545
1,2576336,3315,Sassuolo,3158,Roma,0,1,False,True,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,182.818182,186.454545
7,2576337,3204,SPAL,3164,Sampdoria,3,1,True,False,2,...,18.543724,20.421676,23.525321,20.823186,23.910118,22.417150,19.994541,20.055936,184.818182,182.818182


In [56]:
match_list = []
for matchId in df_matches['game_id']:
    match = [matchId] + getStartingLineupAverageAge(matchesMap[matchId], playerMap)
    match_list.append(match)

In [57]:
df_age = pd.DataFrame(match_list, columns=['game_id', 'team1_id', 'team1_age', 'team2_id', 'team2_age'])

In [58]:
df_matches = df_matches.merge(df_age, on=['game_id', 'team1_id', 'team2_id'])

In [59]:
df_matches.sort_values(by=['game_id'])

,game_id,team1_id,team1_name,team2_id,team2_name,team1_goals,team2_goals,team1_win,team2_win,team1_shot_5,...,team2_averagePassLength_70,team2_averagePassLength_75,team2_averagePassLength_80,team2_averagePassLength_85,team2_averagePassLength_90,team2_averagePassLength_90+,team1_height,team2_height,team1_age,team2_age
377,2575959,3172,Atalanta,3158,Roma,0,1,False,True,1,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,184.272727,184.363636,25.909091,28.181818
370,2575960,3166,Bologna,3185,Torino,1,1,False,False,1,...,26.764134,16.372781,26.406895,25.769032,24.537232,21.045808,183.545455,180.818182,27.545455,27.272727
371,2575961,3197,Crotone,3157,Milan,0,3,False,True,0,...,23.052535,25.086776,27.139089,21.367198,24.603859,22.996558,184.000000,183.363636,25.454545,22.818182
378,2575962,3194,Hellas Verona,3187,Napoli,1,3,False,True,0,...,18.184871,21.658211,23.340103,22.544754,19.788647,23.463808,179.818182,182.636364,26.545455,26.181818
372,2575963,3161,Internazionale,3176,Fiorentina,3,0,True,False,0,...,26.412254,30.426573,28.270499,27.721299,22.617753,25.038108,182.454545,183.727273,28.000000,25.363636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,2576334,3159,Juventus,3194,Hellas Verona,2,1,True,False,2,...,22.495837,18.345308,21.639595,25.619733,26.232859,29.598672,182.181818,181.454545,29.909091,25.636364
0,2576335,3162,Lazio,3161,Internazionale,2,3,False,True,0,...,25.837245,28.349498,27.631560,29.889039,20.469223,21.807284,184.363636,183.454545,26.000000,27.454545
1,2576336,3315,Sassuolo,3158,Roma,0,1,False,True,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,182.818182,186.454545,27.181818,27.636364
7,2576337,3204,SPAL,3164,Sampdoria,3,1,True,False,2,...,23.525321,20.823186,23.910118,22.417150,19.994541,20.055936,184.818182,182.818182,26.000000,24.090909


In [60]:
# # Reset DataFrame with columns in desired order
# cols = ['game_id','team1_name','team1_id','team2_name','team2_id']
# cols.extend(['team1_goals', 'team2_goals', 'team1_win', 'team2_win'])
# cols.extend(['team1_height', 'team2_height'])


# event_names = ['shot', 'corner', 'freeKickShots', 'redCard', 'yellowCard', 'offside']

# cols.extend([f'team1_{name}' for name in event_names])
# cols.extend([f'team2_{name}' for name in event_names])

# cols.extend(['shot', 'corner', 'freeKickShots', 'redCard', 'yellowCard', 'offside'])

# df_matches = df_matches[cols]

In [61]:
df_matches

,game_id,team1_id,team1_name,team2_id,team2_name,team1_goals,team2_goals,team1_win,team2_win,team1_shot_5,...,team2_averagePassLength_70,team2_averagePassLength_75,team2_averagePassLength_80,team2_averagePassLength_85,team2_averagePassLength_90,team2_averagePassLength_90+,team1_height,team2_height,team1_age,team2_age
0,2576335,3162,Lazio,3161,Internazionale,2,3,False,True,0,...,25.837245,28.349498,27.631560,29.889039,20.469223,21.807284,184.363636,183.454545,26.000000,27.454545
1,2576336,3315,Sassuolo,3158,Roma,0,1,False,True,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,182.818182,186.454545,27.181818,27.636364
2,2576329,3173,Cagliari,3172,Atalanta,1,0,True,False,0,...,30.484495,23.834016,23.423180,23.248893,28.366314,24.307511,182.363636,184.090909,26.636364,25.454545
3,2576330,3165,Chievo,3219,Benevento,1,0,True,False,0,...,22.776704,23.801477,23.571837,22.910578,27.238948,24.446687,182.363636,181.636364,31.272727,27.090909
4,2576331,3163,Udinese,3166,Bologna,1,0,True,False,0,...,22.627458,20.203752,22.887248,23.017847,19.316512,33.495381,185.545455,185.454545,27.363636,27.727273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2575967,3315,Sassuolo,3193,Genoa,0,0,False,False,2,...,23.856341,29.320651,25.030822,29.356753,25.875036,26.365318,182.090909,183.000000,27.181818,28.000000
376,2575968,3163,Udinese,3165,Chievo,1,2,False,True,1,...,19.591610,19.064234,20.356348,13.284922,26.320393,27.533136,186.090909,183.090909,25.363636,31.363636
377,2575959,3172,Atalanta,3158,Roma,0,1,False,True,1,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,184.272727,184.363636,25.909091,28.181818
378,2575962,3194,Hellas Verona,3187,Napoli,1,3,False,True,0,...,18.184871,21.658211,23.340103,22.544754,19.788647,23.463808,179.818182,182.636364,26.545455,26.181818


In [62]:
intervals = [
    ("5", (0,5)), 
    ("10", (5,10)), 
    ("15", (15, 20)), 
    ("20", (20, 25)),
    ("25", (25, 30)),
    ("30", (30, 35)),
    ("35", (35, 40)), 
    ("40", (40, 45)),
    ("45", (90, 95)),
    ("50", (95, 100)),
    ("55", (100, 105)),
    ("60", (105, 110)),
    ("65", (110, 115)), 
    ("70", (115, 120)),
    ("75", (120, 125)),
    ("80", (125, 130)),
    ("85", (130, 200))
]

dict_acc = {'game_id': [], 'team1_id': [], 'team2_id': []}
dict_invasion = {'game_id': [], 'team1_id': [], 'team2_id': []}

for idx, row in enumerate(df_matches.iterrows()):
    game_id = row[1].game_id
    team1_id = int(row[1].team1_id)
    team2_id = int(row[1].team2_id)
    
    a_match = []
    for ev in events:
        if ev['matchId'] == game_id:
            a_match.append(ev)

    list_invasion, list_acceleration = get_invasion_index(a_match,match_id,lst=True)

    dict_acc['game_id'].append(game_id)
    dict_invasion['game_id'].append(game_id)

    team_ids = [team1_id, team2_id]

    for team_id, team_label in zip(team_ids, ['team1', 'team2']):
        dict_acc[f'{team_label}_id'].append(team_id)
        dict_invasion[f'{team_label}_id'].append(team_id)
       
        df_acceleration = pd.DataFrame(list_acceleration[team_id], columns=['time','acceleration'])
        df_acceleration['time'] = [x/60. for x in df_acceleration.time]

        df_invasion = pd.DataFrame(list_invasion[team_id],columns=['time','invasion'])
        df_invasion['time'] = [x/60. for x in df_invasion.time]

        for intv_label, (start, end) in intervals:
            
            cond = ((df_acceleration['time'] > start) & (df_acceleration['time'] < end))
            within_interval = df_acceleration.loc[cond]

            key_1 = f'{team_label}_acc_{intv_label}'

            acc_mean = within_interval['acceleration'].mean()
            if np.isnan(acc_mean):
                acc_mean = 0

            if key_1 in dict_acc:
                dict_acc[key_1].append(acc_mean)
            else:
                dict_acc[key_1] = [acc_mean]

            
            cond = ((df_invasion['time'] > start) & (df_invasion['time'] < end))
            within_interval = df_invasion.loc[cond]

            key_2 = f'{team_label}_invasion_{intv_label}'

            invasion_mean = within_interval['invasion'].mean()
            if np.isnan(invasion_mean):
                invasion_mean = 0

            if key_2 in dict_invasion:
                dict_invasion[key_2].append(invasion_mean)
            else:
                dict_invasion[key_2] = [invasion_mean]

acceleration -- nan found
acceleration -- nan found
acceleration -- nan found
acceleration -- nan found
invasion -- nan found
acceleration -- nan found
invasion -- nan found
acceleration -- nan found
invasion -- nan found
acceleration -- nan found
invasion -- nan found
acceleration -- nan found
acceleration -- nan found
invasion -- nan found
acceleration -- nan found
acceleration -- nan found
invasion -- nan found
acceleration -- nan found
invasion -- nan found
acceleration -- nan found
invasion -- nan found
acceleration -- nan found
invasion -- nan found
acceleration -- nan found
invasion -- nan found
acceleration -- nan found
acceleration -- nan found
acceleration -- nan found
acceleration -- nan found
acceleration -- nan found
invasion -- nan found
acceleration -- nan found
invasion -- nan found
acceleration -- nan found
invasion -- nan found
acceleration -- nan found
acceleration -- nan found
acceleration -- nan found
invasion -- nan found
acceleration -- nan found
invasion -- nan 

In [227]:
df_acc = pd.DataFrame(data=dict_acc)
df_acc = df_acc.astype({'team1_id': str, 'team2_id': str})
df_matches = df_matches.merge(df_acc, on=['game_id', 'team1_id', 'team2_id'])

In [228]:
df_inv = pd.DataFrame(data=dict_invasion)
df_inv = df_inv.astype({'team1_id': str, 'team2_id': str})
df_matches = df_matches.merge(df_inv, on=['game_id', 'team1_id', 'team2_id'])

In [229]:
df_matches

,game_id,team1_id,team1_name,team2_id,team2_name,team1_goals,team2_goals,team1_win,team2_win,team1_shot_5,...,team2_invasion_40,team2_invasion_45,team2_invasion_50,team2_invasion_55,team2_invasion_60,team2_invasion_65,team2_invasion_70,team2_invasion_75,team2_invasion_80,team2_invasion_85
0,2576335,3162,Lazio,3161,Internazionale,2,3,False,True,0,...,0.181500,0.171111,0.235909,0.475000,0.205217,0.206429,0.336000,0.456364,0.359333,0.159615
1,2576336,3315,Sassuolo,3158,Roma,0,1,False,True,0,...,0.307059,0.292381,0.245625,0.193750,0.212500,0.361000,0.225333,0.297778,0.374615,0.348182
2,2576329,3173,Cagliari,3172,Atalanta,1,0,True,False,0,...,0.234444,0.181765,0.385294,0.362857,0.202963,0.270000,0.274118,0.540769,0.314737,0.478571
3,2576330,3165,Chievo,3219,Benevento,1,0,True,False,0,...,0.217083,0.004444,0.278182,0.295000,0.062000,0.286111,0.171667,0.311000,0.215000,0.299286
4,2576331,3163,Udinese,3166,Bologna,1,0,True,False,0,...,0.246500,0.281000,0.362857,0.144286,0.383571,0.267647,0.140909,0.266842,0.255263,0.229302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2575967,3315,Sassuolo,3193,Genoa,0,0,False,False,2,...,0.009167,0.357143,0.253750,0.533750,0.208000,0.170833,0.379412,0.170952,0.320000,0.246316
376,2575968,3163,Udinese,3165,Chievo,1,2,False,True,1,...,0.254783,0.161818,0.258333,0.200435,0.326429,0.283889,0.241429,0.215333,0.271333,0.262432
377,2575959,3172,Atalanta,3158,Roma,0,1,False,True,1,...,0.181875,0.265185,0.300000,0.204000,0.373182,0.280476,0.080000,0.106923,0.141333,0.215313
378,2575962,3194,Hellas Verona,3187,Napoli,1,3,False,True,0,...,0.444000,0.138182,0.283333,0.561000,0.240909,0.232381,0.253750,0.405455,0.303000,0.242593


In [230]:
df_matches.to_csv(f'data/final_dfs/{country}_adv.csv', index=False)